# 作业
把提供的 target encoding 代码改为 cython 代码并比较速度区别（如可以实现并行可加分）

In [10]:
import numpy as np
import pandas as pd

输入测试数据

In [ ]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

###初始实现
暴力对每个样本，计算去除该样本后，按ｘ类型的y值均值，整体思想的时间复杂度$O(n^2)$

In [11]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count']) # exclude itself then groupby
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')] # find the average
    return result

In [12]:
%%timeit
result_1 = target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 26.2 s per loop


### 算法优化
课上老师提供的python优化方案，统一预处理计算出每个键的和和个数，再统一计算去除样本后的求均值过程。不算map和dataframe的实现问题，该程序的整体思想的时间复杂度为$O(n)$

我认为在算法思想层面已经没有优化空间了，最优就是$O(n)$

In [13]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict() # calculate the sum for each key
    count_dict = dict() # count the num for each key
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1) # exclude itself then average
    return result

In [14]:
# check whether it is correct (v2==v1)
result_1 = target_mean_v1(data, 'y', 'x')
result_2 = target_mean_v2(data, 'y', 'x')
diff = np.linalg.norm(result_1 - result_2)
print(diff)

0.0


#### 实验结果
性能 26.2s -> 265ms, 性能提升100倍左右

In [15]:
%%timeit
result = target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 265 ms per loop


### Python实现优化
比较重的数据结构就是pandas dataframe和dict，这两个不知道能否优化

a. pandas DataFrame是在操作过程中并不需要，可以预先去取x和y，不知道能否优化
b. map 要查两次，一个键对两个值试试看看

In [19]:
def target_mean_v3_1(data, y_name, x_name):
    n = data.shape[0]
    result = np.zeros(n)
    x_values = data[x_name].values
    y_values = data[y_name].values
    value_dict = dict() # calculate the sum for each key
    count_dict = dict() # count the num for each key
    for i in range(n):
        if x_values[i] not in value_dict.keys():
            value_dict[x_values[i]] = y_values[i]
            count_dict[x_values[i]] = 1
        else:
            value_dict[x_values[i]] += y_values[i]
            count_dict[x_values[i]] += 1
    for i in range(n):
        result[i] = (value_dict[x_values[i]] - y_values[i]) / (count_dict[x_values[i]] - 1) # exclude itself then average
    return result

In [18]:
def target_mean_v3_2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict() # calculate the sum for each key
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = (data.loc[i, y_name], 1)
        else:
            tmp = value_dict[data.loc[i, x_name]]
            value_dict[data.loc[i, x_name]] = (tmp[0] + data.loc[i, y_name], tmp[1]+1)
    for i in range(data.shape[0]):
        tmp = value_dict[data.loc[i, x_name]]
        result[i] = (tmp[0] - data.loc[i, y_name]) / (tmp[1] - 1) # exclude itself then average
    return result

In [34]:
def target_mean_v3_3(data, y_name, x_name):
    n = data.shape[0]
    result = np.zeros(n)
    x_values = data[x_name].values
    y_values = data[y_name].values
    value_dict = dict() # calculate the sum for each key
    for i in range(n):
        if x_values[i] not in value_dict.keys():
            value_dict[x_values[i]] = (y_values[i], 1)
        else:
            tmp = value_dict[x_values[i]]
            value_dict[x_values[i]] = (tmp[0] + y_values[i], tmp[1] + 1)
    for i in range(n):
        tmp = value_dict[x_values[i]]
        result[i] = (tmp[0] - y_values[i]) / (tmp[1] - 1) # exclude itself then average
    return result

In [25]:
# check whether it is correct (v2==v1)
result_2 = target_mean_v2(data, 'y', 'x')
result_3_1 = target_mean_v3_1(data, 'y', 'x')
result_3_2 = target_mean_v3_2(data, 'y', 'x')
result_3_3 = target_mean_v3_3(data, 'y', 'x')
diff_1 = np.linalg.norm(result_3_1 - result_2)
print(diff_1)
diff_2 = np.linalg.norm(result_3_2 - result_2)
print(diff_2)
diff_3 = np.linalg.norm(result_3_3 - result_2)
print(diff_3)

0.0
0.0
0.0


#### 实验结果
1. 3-1 性能 265ms->10.4ms, 性能提升26倍左右
2. 3-2 性能 265ms->237ms, 性能提升10%
3. 3-3(3-1+3-2) 性能 265ms->9.21ms 性能提升28倍左右

In [26]:
%timeit -n 100 target_mean_v3_1(data, 'y', 'x')
%timeit -n 100 target_mean_v3_2(data, 'y', 'x')
%timeit -n 100 target_mean_v3_3(data, 'y', 'x')

100 loops, best of 3: 10.4 ms per loop
100 loops, best of 3: 237 ms per loop
100 loops, best of 3: 9.21 ms per loop


###Cython实现优化
1. 4-0 啥不做直接换cython
2. 4-1 将除了dict都优化成cython中类似c的类型
    - cython定义 n，result，data[x_name].values，data[_name].values
    - data[x_name].values原先是long，转成int
    - for循环修改成cython风格
    - (疑问) 为什么不能试用cnp
3. 4-2 dict替换为libcpp中的unordered_map
   - https://github.com/cython/cython/tree/master/Cython/Includes/libcpp (增加--cplus修饰符，增加c++相关的库)
4. 4-3 (不具有扩展性) unordered_map 替换为数组，因为测试用例x作为键是0-9的整数

In [65]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [46]:
%%cython -a
import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4_0(data, y_name, x_name):
    n = data.shape[0]
    result = np.zeros(n)
    x_values = data[x_name].values
    y_values = data[y_name].values
    value_dict = dict() # calculate the sum for each key
    for i in range(n):
        if x_values[i] not in value_dict.keys():
            value_dict[x_values[i]] = (y_values[i], 1)
        else:
            tmp = value_dict[x_values[i]]
            value_dict[x_values[i]] = (tmp[0] + y_values[i], tmp[1] + 1)
    for i in range(n):
        tmp = value_dict[x_values[i]]
        result[i] = (tmp[0] - y_values[i]) / (tmp[1] - 1) # exclude itself then average
    return result

In [99]:
%%cython -a
import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4_1(data, y_name, x_name):
    cdef int n = data.shape[0]
    cdef double[:] result = np.zeros(n)
    cdef int[:] x_values = data[x_name].values.astype('int32') # 返回的是long类型
    cdef int[:] y_values = data[y_name].values.astype('int32')
    value_dict = dict() # calculate the sum for each key
    for i from 0 <= i < n by 1:
        if x_values[i] not in value_dict.keys():
            value_dict[x_values[i]] = (y_values[i], 1)
        else:
            tmp = value_dict[x_values[i]]
            value_dict[x_values[i]] = (tmp[0] + y_values[i], tmp[1] + 1)
    for i from 0 <= i < n by 1:
        tmp = value_dict[x_values[i]]
        result[i] = (tmp[0] - y_values[i]) / (tmp[1] - 1) # exclude itself then average
    return result

In [98]:
%%cython --cplus -a
import numpy as np
from libcpp.unordered_map cimport unordered_map
from libcpp.pair cimport pair
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4_2(data, y_name, x_name):
    cdef int n = data.shape[0]
    cdef double[:] result = np.zeros(n)
    cdef int[:] x_values = data[x_name].values.astype('int32') # 返回的是long类型
    cdef int[:] y_values = data[y_name].values.astype('int32')
    cdef unordered_map[int, pair[long, int]] value_dict = unordered_map[int, pair[long, int]]()
    for i from 0 <= i < n by 1:
        if value_dict.find(x_values[i]) == value_dict.end():
            value_dict[x_values[i]] = (y_values[i], 1)
        else:
            tmp = value_dict[x_values[i]]
            value_dict[x_values[i]] = (tmp.first + y_values[i], tmp.second + 1)
    for i from 0 <= i < n by 1:
        tmp = value_dict[x_values[i]]
        result[i] = (tmp.first - y_values[i]) / (tmp.second - 1) # exclude itself then average
    return result

In [125]:
%%cython --cplus -a
import numpy as np
from libcpp.unordered_map cimport unordered_map
from libcpp.pair cimport pair
from libcpp.vector cimport vector
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4_3(data, y_name, x_name):
    cdef int n = data.shape[0]
    cdef double[:] result = np.zeros(n)
    cdef int[:] x_values = data[x_name].values.astype('int32') # 返回的是long类型
    cdef int[:] y_values = data[y_name].values.astype('int32')
    cdef int[10] value_dict
    cdef int[10] cnt_dict
    cdef int i
    for i from 0 <= i < 10 by 1:
        value_dict[i]=cnt_dict[i]=0
    for i from 0 <= i < n by 1:
        value_dict[x_values[i]] += y_values[i]
        cnt_dict[x_values[i]] += 1
    for i from 0 <= i < n by 1:
        result[i] = (value_dict[x_values[i]] - y_values[i]) / (cnt_dict[x_values[i]] - 1) # exclude itself then average
    return result

In [147]:
# check whether it is correct (v2==v1)
result_3 = target_mean_v3_3(data, 'y', 'x')
result_4_0 = target_mean_v4_0(data, 'y', 'x')
result_4_1 = target_mean_v4_1(data, 'y', 'x')
result_4_2 = target_mean_v4_2(data, 'y', 'x')
result_4_3 = target_mean_v4_3(data, 'y', 'x')
diff_0 = np.linalg.norm(result_3 - result_4_0)
print(diff_0)
diff_1 = np.linalg.norm(result_3 - result_4_1)
print(diff_1)
diff_2 = np.linalg.norm(result_3 - result_4_2)
print(diff_2)
diff_3 = np.linalg.norm(result_3 - result_4_3)
print(diff_3)

0.0
0.0
0.0
0.0


#### 实验结果
- 4-0 性能 9.21ms->7.97ms, 性能提升15%
- 4-1 性能 9.21ms->1.15ms, 性能提升8倍
- 4-2 性能 9.21ms->457us, 性能提升20倍
- 4-3 性能 9.21ms->39.5us, 性能提升233倍

In [111]:
%timeit -n 100 target_mean_v4_0(data, 'y', 'x')
%timeit -n 100 target_mean_v4_1(data, 'y', 'x')
%timeit -n 100 target_mean_v4_2(data, 'y', 'x')
%timeit -n 100 target_mean_v4_3(data, 'y', 'x')

100 loops, best of 3: 7.61 ms per loop
100 loops, best of 3: 1.15 ms per loop
100 loops, best of 3: 445 µs per loop
100 loops, best of 3: 39.5 µs per loop


### 加分项目 (openmp并行化)
基于v4-3实现
- 5-1 第二个循环求值部分并行化
- 5-2 第一个累计循环并行化 (to do)


In [149]:
%%cython --cplus -a
import numpy as np
from cython.parallel import prange
cimport numpy as cnp
cimport cython


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v5_1(data, y_name, x_name):
    cdef int i
    cdef int n = data.shape[0]
    cdef double[:] result = np.zeros(n)
    cdef int[:] x_values = data[x_name].values.astype('int32') # 返回的是long类型
    cdef int[:] y_values = data[y_name].values.astype('int32')
    cdef int[10] value_dict
    cdef int[10] cnt_dict
    for i from 0 <= i < 10 by 1:
        value_dict[i]=cnt_dict[i]=0
    for i from 0 <= i < n by 1:
        value_dict[x_values[i]] += y_values[i]
        cnt_dict[x_values[i]] += 1
    for i in prange(n, nogil=True):
        result[i] = (value_dict[x_values[i]] - y_values[i]) / (cnt_dict[x_values[i]] - 1)
    return result

In [150]:
# check whether it is correct (v2==v1)
result_3 = target_mean_v3_3(data, 'y', 'x')
result_5_1 = target_mean_v5_1(data, 'y', 'x')
diff_1 = np.linalg.norm(result_3 - result_5_1)
print(diff_1)

0.0


In [151]:
%timeit -n 100 target_mean_v5_1(data, 'y', 'x')

100 loops, best of 3: 41 µs per loop
